<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек-и-подготовка-окружения" data-toc-modified-id="Импорт-библиотек-и-подготовка-окружения-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек и подготовка окружения</a></span></li><li><span><a href="#1.-Подготовка" data-toc-modified-id="1.-Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>1. Подготовка</a></span><ul class="toc-item"><li><span><a href="#1.1-Загрузим-данные" data-toc-modified-id="1.1-Загрузим-данные-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>1.1 Загрузим данные</a></span></li><li><span><a href="#1.2-Произведем-токенизацию-и-превращение-предложений-в-эмбеддинги-при-помощи-BERT" data-toc-modified-id="1.2-Произведем-токенизацию-и-превращение-предложений-в-эмбеддинги-при-помощи-BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>1.2 Произведем токенизацию и превращение предложений в эмбеддинги при помощи BERT</a></span></li><li><span><a href="#1.3-Разделим-данные-на-тренировочную-и-тестовую-выборку." data-toc-modified-id="1.3-Разделим-данные-на-тренировочную-и-тестовую-выборку.-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>1.3 Разделим данные на тренировочную и тестовую выборку.</a></span></li></ul></li><li><span><a href="#2.-Обучение" data-toc-modified-id="2.-Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>2. Обучение</a></span><ul class="toc-item"><li><span><a href="#2.1-Модель-логистической-регрессии" data-toc-modified-id="2.1-Модель-логистической-регрессии-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>2.1 Модель логистической регрессии</a></span></li><li><span><a href="#2.2-Модель-случайного-леса" data-toc-modified-id="2.2-Модель-случайного-леса-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>2.2 Модель случайного леса</a></span></li><li><span><a href="#2.3-Модель-градиентного-бустинга" data-toc-modified-id="2.3-Модель-градиентного-бустинга-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>2.3 Модель градиентного бустинга</a></span></li><li><span><a href="#2.4-Нейронная-сеть" data-toc-modified-id="2.4-Нейронная-сеть-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>2.4 Нейронная сеть</a></span></li><li><span><a href="#2.5-Резюме" data-toc-modified-id="2.5-Резюме-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>2.5 Резюме</a></span></li></ul></li><li><span><a href="#3.-Подбор-гиперпараметров" data-toc-modified-id="3.-Подбор-гиперпараметров-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>3. Подбор гиперпараметров</a></span></li><li><span><a href="#4.-Проверка-на-тестовой-выборке" data-toc-modified-id="4.-Проверка-на-тестовой-выборке-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>4. Проверка на тестовой выборке</a></span></li><li><span><a href="#5.-Выводы" data-toc-modified-id="5.-Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>5. Выводы</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

## Импорт библиотек и подготовка окружения

In [ ]:
!pip install transformers
!pip install catboost
!pip install optuna

In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import torch as t
import pickle
import math
import transformers as tfs
from tqdm import notebook
from tqdm import tqdm

from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, ParameterGrid, StratifiedKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import optuna

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
from torch import cuda
device = t.device("cuda")

In [5]:
# Зафиксируем random seed
random.seed(0)
np.random.seed(0)
t.manual_seed(0)
t.cuda.manual_seed(0)

## 1. Подготовка



### 1.1 Загрузим данные

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/toxic_comments.csv', index_col=0)
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
# Посмотрим информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [ ]:
# Посмотрим на соотношение классов
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Классы несбалансированы.

### 1.2 Произведем токенизацию и превращение предложений в эмбеддинги при помощи BERT

In [ ]:
tokenizer = tfs.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = tfs.DistilBertModel.from_pretrained('distilbert-base-uncased')
model.to(device)

In [ ]:
tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True))

max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(159292, 512)

In [ ]:
### Эта ячейка предназначена для сохранения и загрузки промежуточных результатов ###

# with open('/content/drive/MyDrive/Colab Notebooks/padded.pkl', 'wb') as f:
#             pickle.dump(padded, f)

# with open('/content/drive/MyDrive/Colab Notebooks/attention_mask.pkl', 'wb') as f:
#             pickle.dump(attention_mask, f)

# padded = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/padded.pkl', 'rb'))
# attention_mask = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/attention_mask.pkl', 'rb'))

In [ ]:
input_ids = t.tensor(padded).to(device)
attention_mask = t.tensor(attention_mask).to(device)

In [ ]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(input_ids.shape[0] // batch_size + 1)):
        batch = input_ids[batch_size*i:batch_size*(i+1)]
        attention_mask_batch = attention_mask[batch_size*i:batch_size*(i+1)]
        
        with t.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        t.cuda.empty_cache()

features = np.concatenate(embeddings)
features.shape

(159292, 768)

In [7]:
### Эта ячейка предназначена для сохранения и загрузки промежуточных результатов ###

# with open('/content/drive/MyDrive/Colab Notebooks/features.pkl', 'wb') as f:
#             pickle.dump(features, f)
# features = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/features.pkl', 'rb'))

### 1.3 Разделим данные на тренировочную и тестовую выборку.

In [8]:
train_X, test_X, train_y, test_y = train_test_split(features, data['toxic'].values, test_size=0.1, stratify=data['toxic'], random_state=0)

print('Размер получившихся выборок:')
[print(n, d.shape) for n, d in zip(['train_X', 'test_X', 'train_y', 'test_y'], [train_X, test_X, train_y, test_y])]
print('-'*10)
print('Соотношение классов:')
[print(n, d.value_counts(normalize=True), '\n') for n, d in zip(['train_y\n', 'test_y\n'], [pd.Series(train_y), pd.Series(test_y)])]

Размер получившихся выборок:
train_X (143362, 768)
test_X (15930, 768)
train_y (143362,)
test_y (15930,)
----------
Соотношение классов:
train_y
 0    0.89839
1    0.10161
dtype: float64 

test_y
 0    0.898368
1    0.101632
dtype: float64 



[None, None]

In [9]:
train_X_short, val_X, train_y_short, val_y = train_test_split(train_X, train_y, test_size=0.1, stratify=train_y, random_state=0)

print('Размер получившихся выборок:')
[print(n, d.shape) for n, d in zip(['train_X_short', 'val_X', 'train_y_short', 'val_y'], [train_X_short, val_X, train_y_short, val_y])]
print('-'*10)
print('Соотношение классов:')
[print(n, d.value_counts(normalize=True), '\n') for n, d in zip(['train_y_short\n', 'val_y\n'], [pd.Series(train_y_short), pd.Series(val_y)])]

Размер получившихся выборок:
train_X_short (129025, 768)
val_X (14337, 768)
train_y_short (129025,)
val_y (14337,)
----------
Соотношение классов:
train_y_short
 0    0.898392
1    0.101608
dtype: float64 

val_y
 0    0.898375
1    0.101625
dtype: float64 



[None, None]

In [10]:
# Класс для создания датасета
class CustomDataset(Dataset):
  def __init__(self, data, labels):
    self.labels = labels
    self.data = data
    
  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    label = self.labels[idx]
    data = self.data[idx]
    sample = {"data": data, "labels": label}
    return sample

In [11]:
# Создаем датасеты
dataset_train = CustomDataset(t.FloatTensor(train_X_short), t.FloatTensor(train_y_short))
dataset_valid = CustomDataset(t.FloatTensor(val_X), t.FloatTensor(val_y))
dataset_test = CustomDataset(t.FloatTensor(test_X), t.FloatTensor(test_y))

## 2. Обучение

### 2.1 Модель логистической регрессии

In [ ]:
lr = LogisticRegression(random_state=0)
lr_cv_score = cross_val_score(lr, train_X, train_y, scoring='f1', cv=10, error_score='raise')

In [ ]:
print(f"Logistic Regression F1 MEAN {lr_cv_score.mean()}")
print(f"Logistic Regression F1 STD {lr_cv_score.std()}")

Logistic Regression F1 MEAN 0.7421843352358327
Logistic Regression F1 STD 0.006768508313833861


P.S. Class-weight показывает качество хуже

### 2.2 Модель случайного леса

In [ ]:
rf = RandomForestClassifier(random_state=0)
rf_cv_score = cross_val_score(rf, train_X, train_y, scoring='f1', cv=10, error_score='raise')

In [ ]:
print(f"Random Forest F1 MEAN {rf_cv_score.mean()}")
print(f"Random Forest F1 STD {rf_cv_score.std()}")

Random Forest F1 MEAN 0.6007717571330489
Random Forest F1 STD 0.008720879041619677


### 2.3 Модель градиентного бустинга

In [ ]:
cb = CatBoostClassifier(verbose=False)
cb_cv_score = cross_val_score(cb, train_X, train_y, scoring='f1', cv=10, error_score='raise')

In [ ]:
print(f"CatBoost F1 MEAN {cb_cv_score.mean()}")
print(f"CatBoost F1 STD {cb_cv_score.std()}")

CatBoost F1 MEAN 0.7259454956233083
CatBoost F1 STD 0.006584851944233039


### 2.4 Нейронная сеть

Оценим результат работы нейронной сети на кросс-валидации.

In [138]:
# Класс для создания модели нейронной сети
class Perceptron(t.nn.Module):
  @property
  def device(self):
      for p in self.parameters():
          return p.device

  def __init__(self, input_dim=768, num_layers=0, 
                hidden_dim=64, activation=t.nn.ReLU(), output_dim=2, p=0.0):
      super(Perceptron, self).__init__()
      
      self.layers = t.nn.Sequential()
      self.output_dim = output_dim
      
      prev_size = input_dim
      for i in range(num_layers):
          if i >= 1:
              self.layers.add_module(f'relu{i}', activation)
              self.layers.add_module(f'dropout{i}', t.nn.Dropout(p=p))
          if i+1 != num_layers:
              self.layers.add_module(f'layer{i}', 
                                    t.nn.Linear(prev_size, hidden_dim))
              self.layers.add_module(f'batchnorm{i}', 
                                    t.nn.BatchNorm1d(hidden_dim))
          else:
              self.layers.add_module(f'layer{i}', 
                                    t.nn.Linear(prev_size, output_dim))
              self.layers.add_module(f'batchnorm{i}', 
                                    t.nn.BatchNorm1d(output_dim))  
          prev_size = hidden_dim

      if output_dim == 1:
          self.layers.add_module('classifier', 
                                t.nn.Sigmoid())
      elif output_dim == 2:
          self.layers.add_module('classifier', 
                                t.nn.Softmax(dim=1))

  def forward(self, input):
      output = self.layers(input)
      if self.output_dim == 1:
          return output
      elif self.output_dim == 2:
          return output[:, 1]

# Функция для инициализации весов
def init_weights(layer):
    if type(layer) == t.nn.Linear: # Проверка, что слой – полносвязный
        t.nn.init.xavier_normal_(layer.weight)

In [116]:
# Класс для ранней остановки в ходе обучения модели
class CustomEarlyStopping():
    """
    Ранняя остановка, чтобы остановить тренировку, когда потеря не улучшается после определенной эпохи.
    """
    def __init__(self, patience=10, min_delta=0.5):
        """
        :param patience: сколько эпох ждать, прежде чем остановиться, когда потеря не улучшается
        :param min_delta: минимальная разница между f1-метриками, если улучшений нет, то остановим обучение
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_metric = None
        self.early_stop = False

    def __call__(self, val_metric):
        if self.best_metric == None:
            self.best_metric = val_metric
        elif val_metric - self.best_metric > self.min_delta:
            self.best_metric = val_metric
            # reset counter if validation loss improves
            self.counter = 0
        elif val_metric - self.best_metric < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")
            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True

In [117]:
# Функция для обучения модели
def trainer(model, datasettrain, datasetvalid, loss_function, optimizer, 
            epochs, check_val_metric, patience, min_delta, batch_size=64):

  model.cuda()
  early_stopping = CustomEarlyStopping(patience=patience, min_delta=min_delta)

  val_X = datasetvalid.data
  val_y = datasetvalid.labels
  train_dataloader = DataLoader(datasettrain, batch_size=batch_size, shuffle=True, drop_last=True)
  
  # Обучаем модель заданное количество эпох
  for epoch in notebook.tqdm(range(epochs)):
    for batch in train_dataloader:
      batch_X = batch['data'].cuda()
      batch_y = batch['temp'].cuda()

      optimizer.zero_grad()

      predictions = model.forward(batch_X).flatten()
      train_loss = loss_function(predictions, batch_y)

      train_loss.backward()
      optimizer.step()

    # Проверяем качество на валидационной выборке
    model.eval()
    val_pred = model.forward(val_X.cuda()).flatten()
    f1 = f1_score(val_y.cpu().numpy().astype('int'), (val_pred.detach().cpu().numpy() > 0.5).astype('int'))

    early_stopping(f1)
    if early_stopping.early_stop:
      print('Early Stoppning!!!')
      break

    # Выводим метрики на экран каждые "check_val_metric" эпох
    if check_val_metric != 0 and (epoch % check_val_metric == 0 or epoch == epochs - 1):
      print(epoch)
      print('train_loss=', train_loss.cpu().detach().numpy()) 
      print('test_F1=', f1)

    t.cuda.empty_cache()  

# Функция для тестировани модели
def testing(model, dataset):
  test_X = dataset.data
  test_y = dataset.labels

  predictions = model.forward(test_X.cuda()).flatten()
  f1 = f1_score(test_y.cpu().numpy().astype('int'), (predictions.detach().cpu().numpy() > 0.5).astype('int'))

  return f1, classification_report(test_y.cpu().numpy().astype('int'), (predictions.detach().cpu().numpy() > 0.5).astype('int'))

In [118]:
# Функция для обучения модели без батчей
def trainer_nobatch(model, datasettrain, datasetvalid, loss_function, optimizer, 
                    epochs, check_val_metric, patience, min_delta):

  model.cuda()
  early_stopping = CustomEarlyStopping(patience=patience, min_delta=min_delta)

  val_X = datasetvalid.data
  val_y = datasetvalid.labels

  train_X = datasettrain.data.cuda()
  train_y = datasettrain.labels.cuda()
  
  # Обучаем модель заданное количество эпох
  for epoch in notebook.tqdm(range(epochs)):
    optimizer.zero_grad()

    predictions = model.forward(train_X).flatten()
    train_loss = loss_function(predictions, train_y)

    train_loss.backward()
    optimizer.step()

    # Проверяем качество на валидационной выборке
    model.eval()
    val_pred = model.forward(val_X.cuda()).flatten()
    f1 = f1_score(val_y.cpu().numpy().astype('int'), (val_pred.detach().cpu().numpy() > 0.5).astype('int'))

    early_stopping(f1)
    if early_stopping.early_stop:
      print('Early Stoppning!!!')
      break

    # Выводим метрики на экран каждые "check_val_metric" эпох
    if check_val_metric != 0 and (epoch % check_val_metric == 0 or epoch == epochs - 1):
      print(epoch)
      print('train_loss=', train_loss.cpu().detach().numpy()) 
      print('test_F1=', f1)

    t.cuda.empty_cache()  

In [119]:
# Функция для кросс-валидации нейронной сети
def cross_validate(model, datasettrain, datasetvalid, cv, epochs=100, check_val_metric=1, learning_rate=1e-4, patience=10, min_delta=0.002, batch_size=64):
  skf = StratifiedKFold(cv, random_state=0, shuffle=True)
  number_of_batch = skf.get_n_splits(datasettrain.data, datasettrain.labels)

  list_of_scores = []
  for train_index, test_index in notebook.tqdm(skf.split(datasettrain.data, datasettrain.labels), 
                                               total=number_of_batch):
    train_X_fold = datasettrain.data[train_index]
    train_y_fold = datasettrain.labels[train_index]
    test_X_fold = datasettrain.data[test_index]
    test_y_fold = datasettrain.labels[test_index]
    
    train_dataset_fold = CustomDataset(train_X_fold, train_y_fold)
    test_dataset_fold = CustomDataset(test_X_fold, test_y_fold)
    
    model_fold = model
    model_fold.apply(init_weights)

    model_fold.train()
    trainer(model=model_fold, 
            datasettrain=train_dataset_fold, 
            datasetvalid=datasetvalid, 
            loss_function=t.nn.BCELoss(), 
            optimizer=t.optim.Adam(model_fold.parameters(), lr=learning_rate), 
            epochs=epochs, 
            check_val_metric=check_val_metric, 
            patience=patience,
            min_delta=min_delta, 
            batch_size=batch_size)
    
    model_fold.eval()
    f1, _ = testing(model_fold, test_dataset_fold)
    list_of_scores.append(f1)
    
  return list_of_scores

In [120]:
# Функция для кросс-валидации нейронной сети без батчей
def cross_validate_nobatch(model, datasettrain, datasetvalid, cv, epochs=100, check_val_metric=1, learning_rate=1e-4, patience=10, min_delta=0.002):
  skf = StratifiedKFold(cv, random_state=0, shuffle=True)
  number_of_batch = skf.get_n_splits(datasettrain.data, datasettrain.labels)

  list_of_scores = []
  for train_index, test_index in notebook.tqdm(skf.split(datasettrain.data, datasettrain.labels), 
                                               total=number_of_batch):
    train_X_fold = datasettrain.data[train_index]
    train_y_fold = datasettrain.labels[train_index]
    test_X_fold = datasettrain.data[test_index]
    test_y_fold = datasettrain.labels[test_index]
    
    train_dataset_fold = CustomDataset(train_X_fold, train_y_fold)
    test_dataset_fold = CustomDataset(test_X_fold, test_y_fold)
    
    model_fold = model
    model_fold.apply(init_weights)

    model_fold.train()
    trainer_nobatch(model=model_fold, 
                    datasettrain=train_dataset_fold, 
                    datasetvalid=datasetvalid, 
                    loss_function=t.nn.BCELoss(), 
                    optimizer=t.optim.Adam(model_fold.parameters(), lr=learning_rate), 
                    epochs=epochs, 
                    check_val_metric=check_val_metric, 
                    patience=patience,
                    min_delta=min_delta)
    
    model_fold.eval()
    f1, _ = testing(model_fold, test_dataset_fold)
    list_of_scores.append(f1)
    
  return list_of_scores

Получим результаты кросс-валидации для нейронной сети.

In [ ]:
nn_cv_score = cross_validate(model=Perceptron(input_dim=768, 
                                              num_layers=10, 
                                              hidden_dim=256, 
                                              output_dim=1, 
                                              p=0.0),
                             datasettrain=dataset_train, 
                             datasetvalid=dataset_valid, 
                             cv=10)

In [ ]:
print(f"NeuralNet F1 MEAN {np.mean(nn_cv_score)}")
print(f"NeuralNet F1 STD {np.std(nn_cv_score)}")

NeuralNet F1 MEAN 0.7317718127224702
NeuralNet F1 STD 0.012598051881490803


Посмотрим также на качество модели, обученной без батчей.

In [ ]:
nn_cv_score_nobatch = cross_validate_nobatch(model=Perceptron(input_dim=768, 
                                                              num_layers=10, 
                                                              hidden_dim=256, 
                                                              output_dim=1, 
                                                              p=0.0),
                                             datasettrain=dataset_train, 
                                             datasetvalid=dataset_valid, 
                                             cv=10,
                                             patience=150, 
                                             epochs=1000, 
                                             check_val_metric=1, 
                                             learning_rate=1e-4, 
                                             min_delta=0.002)

In [ ]:
print(f"NeuralNet #nobatch F1 MEAN {np.mean(nn_cv_score_nobatch)}")
print(f"NeuralNet #nobatch F1 STD {np.std(nn_cv_score_nobatch)}")

NeuralNet #nobatch F1 MEAN 0.7242791469557217
NeuralNet #nobatch F1 STD 0.02025568477783922


### 2.5 Резюме

На данном этапе мы проверили качество четырех моделей на 10-fold кросс-валидации. Были получены следующие результаты:  

|Модель|F1 mean |F1 std|
|-----|-----|-----|
|LogisticRegression|0.742|0.007|
|RandomForest|0.6|0.009|
|CatBoost| 0.726|0.007|
|NeuralNet|0.732|0.01|
|NeuralNet #nobatch|0.724|0.02|   

Следующим шагом подберем гиперпараметры для модели нейронной сети и попробуем улучшить ее результат.

## 3. Подбор гиперпараметров

Для подбора гиперпараметров используем библиотеку Optuna.  
Для ускорения поиска гиперпараметров используем 3-fold кросс-валидацию вместо 10-fold.

In [148]:
def objective(trial):
    
    # Dropout
    p_suggested = trial.suggest_float('p', 0.0, 0.7, step=0.1) 
    
    # Activation
    activation_dict = {"nn.LeakyReLU()": t.nn.LeakyReLU(),
                       "nn.ReLU()": t.nn.ReLU(),
                       "nn.ELU()": t.nn.ELU()
                      }
    activation_suggested = trial.suggest_categorical('activation', ["nn.LeakyReLU()", "nn.ReLU()", "nn.ELU()"])
    activation_suggested = activation_dict.get(activation_suggested)
  
    # Number of hidden layers
    num_layers_suggested = trial.suggest_int('num_layers', 5, 50,  step=5)

    # Number of neurons in hidden dimensions
    hidden_dim_suggested = trial.suggest_int('hidden_dim', 64, 512,  step=64)
    
    # Learning rate
    lr_suggested = trial.suggest_loguniform('learning_rate', 1e-6, 1e-0)
    
    # Batch size 
    batch_size_suggested = trial.suggest_int('batch_size', 64, 1024,  step=64)
    
    # Patience
    patience_suggested = trial.suggest_int('patience', 10, 30,  step=5)
    
    f1_scores = cross_validate(model=Perceptron(input_dim=768, 
                                               num_layers=num_layers_suggested, 
                                               hidden_dim=hidden_dim_suggested, 
                                               activation=activation_suggested,
                                               output_dim=1, 
                                               p=p_suggested), 
                              datasettrain=dataset_train, 
                              datasetvalid=dataset_valid, 
                              cv=3,
                              epochs=500,
                              check_val_metric=1,
                              learning_rate=lr_suggested, 
                              patience=patience_suggested,
                              min_delta=0.002,
                              batch_size=batch_size_suggested)

    return np.mean(f1_scores)

In [ ]:
%%time
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=18000)

In [ ]:
# Посмотрим на наилучшее качество на 3-fold кросс-валидации
study.best_trial.values

[0.753203744456273]

In [146]:
# Посмотрим на подобранные гиперпараметры
best_params = study.best_trial.params
best_params

{'p': 0.5,
 'activation': 'nn.ELU()',
 'num_layers': 5,
 'hidden_dim': 320,
 'learning_rate': 1.670426452626111e-05,
 'batch_size': 1024,
 'patience': 25}

In [150]:
activation_dict = {
    "nn.LeakyReLU()": t.nn.LeakyReLU(),
    "nn.ReLU()": t.nn.ReLU(),
    "nn.ELU()": t.nn.ELU()
}

Посмотрим также какое качество покажет модель с подобранными гиперпараметрами на 10-fold кросс-валидации.

In [ ]:
f1_scores = cross_validate(model=Perceptron(input_dim=768, 
                                            num_layers=best_params['num_layers'], 
                                            hidden_dim=best_params['hidden_dim'], 
                                            activation=activation_dict[best_params['activation']],
                                            output_dim=1, 
                                            p=best_params['p']), 
                          datasettrain=dataset_train, 
                          datasetvalid=dataset_valid, 
                          cv=10,
                          epochs=500,
                          check_val_metric=1,
                          learning_rate=best_params['learning_rate'], 
                          patience=best_params['patience'],
                          min_delta=0.002,
                          batch_size=best_params['batch_size'])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

0
train_loss= 0.6995009
test_F1= 0.3286689025095409
1
train_loss= 0.1617064
test_F1= 0.6214099216710184
2
train_loss= 0.15001813
test_F1= 0.6904761904761906
3
train_loss= 0.1458354
test_F1= 0.7103393843725336
4
train_loss= 0.13430719
test_F1= 0.7232277526395173
INFO: Early stopping counter 1 of 25
5
train_loss= 0.12601247
test_F1= 0.7207843137254902
INFO: Early stopping counter 2 of 25
6
train_loss= 0.11101131
test_F1= 0.7228537828302627
7
train_loss= 0.15160483
test_F1= 0.731614859742229
INFO: Early stopping counter 1 of 25
8
train_loss= 0.14748804
test_F1= 0.7293754818812643
INFO: Early stopping counter 2 of 25
9
train_loss= 0.12900734
test_F1= 0.7294573643410852
INFO: Early stopping counter 3 of 25
10
train_loss= 0.1450806
test_F1= 0.7317817626859977
INFO: Early stopping counter 4 of 25
11
train_loss= 0.15319067
test_F1= 0.7332038834951458
INFO: Early stopping counter 5 of 25
12
train_loss= 0.12120539
test_F1= 0.7309205350118017
INFO: Early stopping counter 6 of 25
13
train_loss= 0.

  0%|          | 0/500 [00:00<?, ?it/s]

0
train_loss= 0.706788
test_F1= 0.32809773123909247
1
train_loss= 0.16735536
test_F1= 0.6151843817787419
2
train_loss= 0.16895813
test_F1= 0.6761019005256773
3
train_loss= 0.14413103
test_F1= 0.6989289964299881
4
train_loss= 0.14710203
test_F1= 0.711651628089447
5
train_loss= 0.14144102
test_F1= 0.7245600612088753
INFO: Early stopping counter 1 of 25
6
train_loss= 0.12433989
test_F1= 0.7159225602528645
7
train_loss= 0.12428923
test_F1= 0.7290076335877862
INFO: Early stopping counter 1 of 25
8
train_loss= 0.13776079
test_F1= 0.7286643704554152
INFO: Early stopping counter 2 of 25
9
train_loss= 0.124412954
test_F1= 0.7272727272727273
10
train_loss= 0.12838155
test_F1= 0.7451708766716196
INFO: Early stopping counter 1 of 25
11
train_loss= 0.13621292
test_F1= 0.7321221492075763
INFO: Early stopping counter 2 of 25
12
train_loss= 0.13881432
test_F1= 0.7377609108159393
INFO: Early stopping counter 3 of 25
13
train_loss= 0.12487112
test_F1= 0.7395793499043977
14
train_loss= 0.12669139
test_F1

...

0
train_loss= 0.7159287
test_F1= 0.3593264248704663
1
train_loss= 0.14984491
test_F1= 0.6422174840085288
2
train_loss= 0.18086308
test_F1= 0.6892700438771441
3
train_loss= 0.12340522
test_F1= 0.7067551737602499
4
train_loss= 0.14464703
test_F1= 0.7101391650099403
5
train_loss= 0.12670779
test_F1= 0.7267843438219496
INFO: Early stopping counter 1 of 25
6
train_loss= 0.11211914
test_F1= 0.7256499805975941
INFO: Early stopping counter 2 of 25
7
train_loss= 0.14668518
test_F1= 0.7266327727806022
INFO: Early stopping counter 3 of 25
8
train_loss= 0.13767081
test_F1= 0.7258444619010213
9
train_loss= 0.15889984
test_F1= 0.7384500745156485
INFO: Early stopping counter 1 of 25
10
train_loss= 0.12816782
test_F1= 0.7305759567066099
INFO: Early stopping counter 2 of 25
11
train_loss= 0.128529
test_F1= 0.7317454119484575
INFO: Early stopping counter 3 of 25
12
train_loss= 0.15006109
test_F1= 0.7269828616978876
INFO: Early stopping counter 4 of 25
13
train_loss= 0.1356271
test_F1= 0.7388535031847134

  0%|          | 0/500 [00:00<?, ?it/s]

0
train_loss= 0.7195058
test_F1= 0.32186759896244505
1
train_loss= 0.14031214
test_F1= 0.6373254337706306
2
train_loss= 0.14035994
test_F1= 0.6761479073547338
3
train_loss= 0.15414613
test_F1= 0.7014925373134328
4
train_loss= 0.12813571
test_F1= 0.7162214479287651
5
train_loss= 0.14608942
test_F1= 0.7195932733672273
6
train_loss= 0.17424977
test_F1= 0.7309451219512195
INFO: Early stopping counter 1 of 25
7
train_loss= 0.13763392
test_F1= 0.7286702536510377
INFO: Early stopping counter 2 of 25
8
train_loss= 0.14144862
test_F1= 0.7290944123314066
9
train_loss= 0.117568426
test_F1= 0.7352264006139678
10
train_loss= 0.12389218
test_F1= 0.741514360313316
INFO: Early stopping counter 1 of 25
11
train_loss= 0.14114837
test_F1= 0.736360167874857
12
train_loss= 0.14257264
test_F1= 0.7472687545520758
INFO: Early stopping counter 1 of 25
13
train_loss= 0.14865988
test_F1= 0.7392614920874152
INFO: Early stopping counter 2 of 25
14
train_loss= 0.14069033
test_F1= 0.7431365174877774
INFO: Early stop

  0%|          | 0/500 [00:00<?, ?it/s]

0
train_loss= 0.70829284
test_F1= 0.33751643360822275
1
train_loss= 0.16115083
test_F1= 0.6305841924398625
2
train_loss= 0.13166872
test_F1= 0.6822958771220695
3
train_loss= 0.1294462
test_F1= 0.7077769049489394
4
train_loss= 0.14177246
test_F1= 0.7195972114639814
INFO: Early stopping counter 1 of 25
5
train_loss= 0.1311328
test_F1= 0.7166666666666667
6
train_loss= 0.13088697
test_F1= 0.727969348659004
7
train_loss= 0.13772362
test_F1= 0.7319311663479924
INFO: Early stopping counter 1 of 25
8
train_loss= 0.13141552
test_F1= 0.7330524703178859
INFO: Early stopping counter 2 of 25
9
train_loss= 0.13905714
test_F1= 0.7270597422881687
INFO: Early stopping counter 3 of 25
10
train_loss= 0.15818696
test_F1= 0.7309486780715395
11
train_loss= 0.15092018
test_F1= 0.7347725520431766
INFO: Early stopping counter 1 of 25
12
train_loss= 0.12432614
test_F1= 0.7359938603223332
13
train_loss= 0.11641824
test_F1= 0.7421431276031807
INFO: Early stopping counter 1 of 25
14
train_loss= 0.12178763
test_F1=

In [ ]:
print(f"NeuralNet #optimized F1 MEAN {np.mean(f1_scores)}")
print(f"NeuralNet #optimized F1 STD {np.std(f1_scores)}")

NeuralNet #optimized F1 MEAN 0.7485824474742111
NeuralNet #optimized F1 STD 0.00845260573141541


Благодаря оптимизации гиперпараметров удалось улучшить качество на кросс-валидации с 0.73 до 0.75 на 3-fold кросс валидации и до 74.9 на 10-fold кросс-валидации. При этом удалось снизить стандартное отклонение с 0.013 до 0.008.  

## 4. Проверка на тестовой выборке

Проверим качество финальной модели на тестовой выборке.

In [ ]:
best_net = Perceptron(input_dim=768, 
                      num_layers=best_params['num_layers'], 
                      hidden_dim=best_params['hidden_dim'], 
                      activation=activation_dict[best_params['activation']],
                      output_dim=1, 
                      p=best_params['p'])

best_net.apply(init_weights)

best_net.train()
trainer(model=best_net, 
        datasettrain=dataset_train, 
        datasetvalid=dataset_valid, 
        loss_function=t.nn.BCELoss(), 
        optimizer=t.optim.Adam(best_net.parameters(), lr=best_params['learning_rate']), 
        epochs=500, 
        check_val_metric=1, 
        patience=10,
        min_delta=0.001, 
        batch_size=best_params['batch_size'])

  0%|          | 0/500 [00:00<?, ?it/s]

0
train_loss= 0.66612285
test_F1= 0.3653395784543326
1
train_loss= 0.1741586
test_F1= 0.6526315789473685
2
train_loss= 0.15229735
test_F1= 0.6878048780487805
3
train_loss= 0.15475649
test_F1= 0.7133411124076235
4
train_loss= 0.13968232
test_F1= 0.7155747351902707
5
train_loss= 0.111509584
test_F1= 0.719652036378015
6
train_loss= 0.13884172
test_F1= 0.7272018706157443
7
train_loss= 0.1665387
test_F1= 0.7285436893203882
8
train_loss= 0.12657876
test_F1= 0.7316698656429943
INFO: Early stopping counter 1 of 10
9
train_loss= 0.15164332
test_F1= 0.7288333983612952
10
train_loss= 0.13961048
test_F1= 0.7407689379520367
INFO: Early stopping counter 1 of 10
11
train_loss= 0.15177232
test_F1= 0.7356948228882834
INFO: Early stopping counter 2 of 10
12
train_loss= 0.11662787
test_F1= 0.7337509788566955
INFO: Early stopping counter 3 of 10
13
train_loss= 0.11404749
test_F1= 0.7397683397683397
14
train_loss= 0.10908087
test_F1= 0.7437185929648241
INFO: Early stopping counter 1 of 10
15
train_loss= 0.

In [ ]:
best_net.eval()
f1, report = testing(best_net, dataset_test)

print(report)

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     14311
           1       0.82      0.70      0.76      1619

    accuracy                           0.95     15930
   macro avg       0.89      0.84      0.86     15930
weighted avg       0.95      0.95      0.95     15930



In [ ]:
f1

0.7552587646076795

Нам удалось получить модель, которая показывает качество на тестовой выборке F1 = 75.5.  
Для того, чтобы получить такой результат, мы немного снизили patience и параметр min_delta. Таким образом, мы позволили сети продолжать обучаться даже при небольшом увеличении качества на валидационной выборке, но при этом за счет сокращения количества итераций без увеличения качества на min_delta, предотвратили переобучение модели.  
Финальная модель способна распознать 70% токсичных комментариев из всех токсичных комментариев, которые есть. При этом, среди тех комментариев, которые она распознала как токсичные, модель ошибается лишь в 18% случаев.

## 5. Выводы

Целью данного проекта являлось построить модель, способную классифицировать комментарии как позитивные и негативные.  
В ходе работы над проектом были проделаны следующие шаги:
- Предложения из корпуса были преобразованы в эмбеддинги при помощи модели BERT
- Было проверено качество работы 4-х моделей на 10-fold кросс-валидации. Были получены следующие результаты:  

|Модель|F1 mean |F1 std|
|-----|-----|-----|
|LogisticRegression|0.742|0.007|
|RandomForest|0.6|0.009|
|CatBoost| 0.726|0.007|
|NeuralNet|0.732|0.01|
|NeuralNet #nobatch|0.724|0.02|   


- Следующим шагом был осуществлен подбор гиперпараметров для модели нейронной сети. На данном этапе удалось улучшить качество модели с 0.73 до 0.75 на 3-fold кросс валидации и до 74.9 на 10-fold кросс-валидации. При этом удалось снизить стандартное отклонение с 0.013 до 0.008.  
- На следующем этапе финальная модель была проверена на тестовом наборе данных. Качество ее работы составило 0.755 по F1-метрике.